In [129]:
import os
import time
import pickle
import pydicom
import pandas as pd
pd.set_option("display.max_colwidth", None)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pydicom.pixel_data_handlers.util import apply_voi_lut

## In each studiesUID, copy bounding boxes from one seriesUID to another seriesUID

In [178]:
df_sequences = pd.read_csv('/home/single3/Documents/tintrung/brainmri/tinnvt/brain-mri-abnormal/csv_new/df_series_clear.csv')
df_2_classes = pd.read_pickle('/home/single3/Documents/tintrung/brainmri/tinnvt/brain-mri-abnormal/csv_new/brain-mri-xml-dataset-2classes.pkl')
df_2_classes = df_2_classes.reset_index()
df_dicom = pd.read_pickle('/home/single3/Documents/tintrung/brainmri/tinnvt/brain-mri-abnormal/csv_new/summary_dicom.pkl')
df_xml = pd.read_pickle('/home/single3/Documents/tintrung/brainmri/tinnvt/summary_xml.pkl')
print(f'df_sequences: {df_sequences.columns}')
print(f'df_2_classes: {df_2_classes.columns}')
print(f'df_dicom: {df_dicom.columns}')
print(f'df_xml: {df_xml.columns}')

df_sequences: Index(['SeriesInstanceUID', 'StudyInstanceUID', 'SeriesLabel'], dtype='object')
df_2_classes: Index(['index', 'studyUid', 'seriesUid', 'imageUid', 'label', 'x1', 'x2', 'y1',
       'y2'],
      dtype='object')
df_dicom: Index(['StudyInstanceUID', 'SeriesInstanceUID', 'SeriesDescription', 'Rows',
       'Columns', 'SliceThickness', 'PixelSpacing', 'SpacingBetweenSlices',
       'SliceLocation', 'SmallestImagePixelValue', 'LargestImagePixelValue',
       'WindowCenter', 'WindowWidth', 'ImagePositionPatient',
       'ImageOrientationPatient', 'RepetitionTime', 'EchoTime',
       'PatientPosition', 'DicomFileName'],
      dtype='object')
df_xml: Index(['xml_name', 'patientPid', 'diagnosis', 'sessionId', 'type',
       'annotation', 'scope', 'pointUnit', 'createTimestamp', 'imageUid',
       'seriesUid', 'studyUid', 'tags', 'point'],
      dtype='object')


In [203]:
df_sequences = df_sequences.rename(columns={'SeriesInstanceUID':'seriesUid', 'StudyInstanceUID':'studyUid'})
df_dicom = df_dicom.rename(columns={'SeriesInstanceUID':'seriesUid', 'StudyInstanceUID':'studyUid'})
# In df_dicom['ImagePositionPatient'] column have None values
z_axis = []
for item in list(df_dicom['ImagePositionPatient']):
    if item != None:
        z_axis.append(item[-1])
    else:
        z_axis.append(None)
df_dicom['z-axis-ImagePositionPatient'] = z_axis
df_dicom['z-axis-ImagePositionPatient']

0        19.06570
1       -46.83690
2        37.03910
3       -22.87230
4         7.08339
           ...   
71847   -27.49980
71848   -35.35790
71849   -43.21590
71850   -51.07400
71851   -58.93200
Name: z-axis-ImagePositionPatient, Length: 71852, dtype: float64

In [197]:
# df_dcm_xml = pd.merge(df_dicom, df_xml, how='left', on=['seriesUid', 'studyUid'])
total_series = list(df_dicom['seriesUid'].drop_duplicates())
total_annoted_studies = list(df_xml['studyUid'].drop_duplicates())
total_annoted_series = list(df_xml['seriesUid'].drop_duplicates())
total_not_annoted_series = list(set(total_series)-set(total_annoted_studies))
print(f'There are {len(total_not_annoted_series)} series that have not annoted!')

# for stu in annoted_studies:
#     # print(f'================== Processing study {stu} ==================')
#     df_study = df_dicom[df_dicom['studyUid']==stu]
#     all_series = list(set(df_study['seriesUid']))
#     annot_seri = list(set(df_xml[df_xml['studyUid']==stu]['seriesUid']))[0]
#     not_annot_series = list(set(all_series)-set(annot_seri))
#     df_annot_seri = df_xml[df_xml['seriesUid']==annot_seri]
#     df_not_annot_seri = df_xml[df_xml['seriesUid']==not_annot_series]
#     points = df_annot_seri['point']
#     print(points)

#     # print(annot_seri)
#     # print(len(not_annot_series))
#     # print(df_annot_seri)
#     print('*'*59)
#     # break

There are 1479 series that have not annoted!


In [198]:
choisen_studies = '1.2.840.113619.6.388.101774739834280643528361334251926593124'
tmp1 = list(set(df_dicom[df_dicom['studyUid']==choisen_studies]['seriesUid']))
tmp1

['1.2.840.113619.2.80.2802896632.11452.1599609724.26.4.2',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.526',
 '1.2.840.113619.2.5.1916222523715996118091599611809219000',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.529',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.530',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.520',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.527',
 '1.2.840.113619.2.388.57473.14165493.11896.1599525311.834',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.528',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.524',
 '1.2.840.113619.2.80.2802896632.11452.1599609703.1.4.2',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.525',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.517']

In [199]:
tmp2 = list(set(df_xml[df_xml['studyUid']==choisen_studies]['seriesUid']))
tmp2

['1.2.840.113619.2.388.57473.14165493.12654.1599525247.529',
 '1.2.840.113619.2.388.57473.14165493.12654.1599525247.526']

In [200]:
a = df_xml[df_xml['seriesUid']=='1.2.840.113619.2.388.57473.14165493.12654.1599525247.526']['z-axis-annotation']
a

0    [-40.168701171875, -4.1686601638794, -18.568700790405]
1     [-36.568698883057, -18.568700790405, 39.031299591064]
2     [17.431299209595, -2.3686599731445, -18.568700790405]
Name: z-axis-annotation, dtype: object

In [201]:
b = df_dicom[df_dicom['seriesUid']=='1.2.840.113619.2.5.1916222523715996118091599611809219000']['z-axis-ImagePositionPatient']
b

0     19.06570
1    -46.83690
2     37.03910
3    -22.87230
4      7.08339
5     13.07450
6    -52.82810
7     55.01260
8     31.04800
9     43.03030
10   -28.86350
11     1.09225
12   -10.89000
13   -58.81920
14    49.02140
15   -34.85460
16    -4.89890
17   -40.84580
44   -16.88120
45    25.05680
Name: z-axis-ImagePositionPatient, dtype: float64